In [1]:
# Load the data.table library
library(data.table)

# Read the CitiBike trip data for September 2018
df <- fread("2018-citibike-tripdata/9_September/201809-citibike-tripdata_1.csv")

df=data.table(df)

In [ ]:
cat("question 1.1 :\n")
print(df[1,4])

cat("question 1.2 :\n")
print(df[1:5,5])

cat("question 1.3 :\n")
print(df[,10:11])

cat("question 1.4 :\n")
print(df[gender==2,14:15])

question 1 :
   start station id
              <num>
1:              252
question 2 :
                start station name
                            <char>
1:    MacDougal St & Washington Sq
2: Cadman Plaza West & Montague St
3:                 1 Ave & E 62 St
4:         St James Pl & Oliver St
5:                 W 13 St & 6 Ave
question 3 :
         end station latitude end station longitude
                        <num>                 <num>
      1:             40.69326             -73.96890
      2:             40.69103             -73.99183
      3:             40.67872             -73.99599
      4:             40.71334             -74.00936
      5:             40.73401             -74.00294
     ---                                           
 999996:             40.76818             -73.95910
 999997:             40.74239             -73.99726
 999998:             40.74475             -73.99915
 999999:             40.74312             -73.98215
1000000:             40.72020   

In [3]:
cat("question 1.5 :\n")
print(df[,.N])

cat("question 1.6 :\n")
print(df[1:10, mean(tripduration)])

cat("question 1.7 :\n")
print(df[,mean(tripduration),by=gender])

question 1.5 :
[1] 1000000
question 1.6 :
[1] 1939.1
question 1.7 :
   gender        V1
    <int>     <num>
1:      1  853.1404
2:      0 2005.6706
3:      2 1016.5074


In [4]:
cat("question 1.8 :\n")
print(df[,.(mean_duration_by_gender=mean(tripduration)),by=gender])

cat("question 1.9 :\n")
print(df[,.(mean_duration=mean(tripduration),number_of_trips=.N),by=gender])


question 1.8 :
   gender mean_duration_by_gender
    <int>                   <num>
1:      1                853.1404
2:      0               2005.6706
3:      2               1016.5074
question 1.9 :
   gender mean_duration number_of_trips
    <int>         <num>           <int>
1:      1      853.1404          651245
2:      0     2005.6706          103169
3:      2     1016.5074          245586


In [24]:
library(fixest)

cat("task 2")

df_short = df[,.(no_of_trips=.N),by= .(`birth year`)]

model = feols(no_of_trips ~ `birth year`, data = df_short)

etable(model)
summary(model)


task 2

,,model
,<chr>,<chr>
1,Dependent Var.:,no_of_trips
2,,
3,Constant,"-530,648.0*** (77,798.0)"
4,`birthyear`,277.2*** (39.89)
5,_______________,________________________
6,S.E. type,IID
7,Observations,100
8,R2,0.33018
9,Adj. R2,0.32334


OLS estimation, Dep. Var.: no_of_trips
Observations: 100
Standard-errors: IID 
                Estimate Std. Error  t value   Pr(>|t|)    
(Intercept)  -530647.952 77797.9766 -6.82085 7.5104e-10 ***
`birth year`     277.244    39.8892  6.95036 4.0687e-10 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 12,892.1   Adj. R2: 0.323343

In [20]:

covariance <- cov(df_short$`birth year`, df_short$no_of_trips)

variance_birth_year <- var(df_short$`birth year`)

# Calculate slope
manual_slope <- covariance / variance_birth_year

cat("Manual calculation of slope (task 3):\n")
cat("Covariance(birth year, no_of_trips):", covariance, "\n")
cat("Variance(birth year):", variance_birth_year, "\n")
cat("Slope (cov/var):", manual_slope, "\n")

cat("\nComparison with regression model:\n")
cat("Model coefficient for birth year:", coef(model)[2], "\n")
cat("Difference:", abs(manual_slope - coef(model)[2]), "\n")


Manual calculation of slope (task 3):
Covariance(birth year, no_of_trips): 298497.6 
Variance(birth year): 1076.66 
Slope (cov/var): 277.244 

Comparison with regression model:
Model coefficient for birth year: 277.244 
Difference: 3.899459e-11 


In [23]:

df_log <- df_short[!is.na(`birth year`) & `birth year` > 0 & no_of_trips > 0]

cat("\nFitting log-log model:\n")
log_model <- feols(log(no_of_trips) ~ log(`birth year`), data = df_log)

cat("Original linear model:\n")
print(summary(model))

cat("\nLog-log model:\n")
print(summary(log_model))

cat("\nInterpretation of log-log model:\n")
cat("The coefficient", round(coef(log_model)[2], 4), "means:\n")
cat("A 1% increase in birth year is associated with a", round(coef(log_model)[2], 4), "% change in number of trips\n")



Fitting log-log model:


Original linear model:
OLS estimation, Dep. Var.: no_of_trips
Observations: 100
Standard-errors: IID 
                Estimate Std. Error  t value   Pr(>|t|)    
(Intercept)  -530647.952 77797.9766 -6.82085 7.5104e-10 ***
`birth year`     277.244    39.8892  6.95036 4.0687e-10 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 12,892.1   Adj. R2: 0.323343

Log-log model:
OLS estimation, Dep. Var.: log(no_of_trips)
Observations: 100
Standard-errors: IID 
                   Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)       -1266.597   72.07492 -17.5733 < 2.2e-16 ***
log(`birth year`)   168.084    9.51421  17.6666 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.58119   Adj. R2: 0.758601

Interpretation of log-log model:
The coefficient 168.0838 means:
A 1% increase in birth year is associated with a 168.0838 % change in number of trips


Task 4: the slop tells us there is a causality between the 2 features, since the slop is positive and not null

# Special Case: Binary Independent Variable

## When X is Binary (0 or 1)

For a binary independent variable X ∈ {0, 1}, the OLS slope formula simplifies dramatically:

### General Formula:
$$\beta_1 = \frac{Cov(X,Y)}{Var(X)} = \frac{\sum(X_i - \bar{X})(Y_i - \bar{Y})}{\sum(X_i - \bar{X})^2}$$

### For Binary X, this becomes:
$$\beta_1 = \bar{Y}_1 - \bar{Y}_0$$

Where:
- $\bar{Y}_1$ = mean of Y when X = 1
- $\bar{Y}_0$ = mean of Y when X = 0

### Why this simplification works:

1. **Mean of binary variable**: $\bar{X} = p$ (proportion of 1s)

2. **Variance of binary variable**: $Var(X) = p(1-p)$

3. **Covariance simplifies to**: $Cov(X,Y) = p(1-p)(\bar{Y}_1 - \bar{Y}_0)$

4. **Therefore**: 
   $$\beta_1 = \frac{p(1-p)(\bar{Y}_1 - \bar{Y}_0)}{p(1-p)} = \bar{Y}_1 - \bar{Y}_0$$

### Interpretation:
The slope coefficient is simply the **difference in means** between the two groups!

This is why regression with a binary independent variable is equivalent to a two-sample t-test for comparing means.


# Causal Interpretation: Does Slope = 277 Imply Causation?

## Short Answer: NO

The slope coefficient of **277** shows **correlation**, not **causation**.


However, we will use the t test and value the p value to assume if there is causality. If p-value is inferior to 5%, we will assume causality by ommiting variable bias, lack of experimental observations etc.


# OLS Problems That Give Correlation (Not Causation)

## Main Issues:

1. **Omitted Variable Bias** - Missing important variables
2. **Reverse Causality** - Y causes X instead of X causes Y  
3. **Selection Bias** - Non-random sample selection
4. **Measurement Error** - Variables measured incorrectly
5. **Simultaneity** - X and Y determined simultaneously
6. **Confounding** - Third variable affects both X and Y
7. **Endogeneity** - X correlated with error term

## Result:
All lead to **correlation without causation** - you get significant coefficients but wrong causal interpretation.
